# SCANIA Predictive Maintenance — Feature Engineering (Train)

This notebook builds a *per-vehicle* feature matrix suitable for machine learning.

We start from the raw training data:

- `train_operational_readouts.csv`
- `train_tte.csv`
- `train_specifications.csv`

Steps:

1. Load data from Azure Data Lake Storage Gen2.
2. Merge operational readouts, TTE (labels), and specifications.
3. Aggregate the variable-length time series per `vehicle_id` into **fixed-size feature vectors**:
   - Counter-based features (e.g., final value, trend, volatility).
   - Histogram-based features (mean, std, min, max per bin).
   - Derived histogram features like total mass and centroid.
4. Create a final feature matrix: one row per `vehicle_id` with the target `in_study_repair`.
5. Save the resulting feature table for use in model training.


### Imports & configuration

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from adlfs import AzureBlobFileSystem

# ---------- CONFIGURATION ----------
ACCOUNT_NAME = "scaniapdmstorage"
FILE_SYSTEM = "scania-dataset"

# ⚠️ IMPORTANT:
# For now we use the storage account key directly for simplicity.
# DO NOT commit this key to GitHub. Later, this can be replaced with managed identity.
ACCOUNT_KEY = "<PASTE-YOUR-STORAGE-ACCOUNT-KEY-HERE>"  # TODO: replace

# Paths inside the filesystem
TRAIN_OP_PATH = "scania-dataset/train/train_operational_readouts.csv"
TRAIN_TTE_PATH = "scania-dataset/train/train_tte.csv"
TRAIN_SPEC_PATH = "scania-dataset/train/train_specifications.csv"

# Keys / column names
VEHICLE_COL = "vehicle_id"
TIME_COL = "time_step"
TARGET_COL = "in_study_repair"

# Known counter columns (single numeric counters)
COUNTER_COLS = [
    "171_0", "666_0", "427_0", "837_0",
    "309_0", "835_0", "370_0", "100_0",
]

# Histogram groups (prefix -> will collect columns starting with this)
HISTOGRAM_PREFIXES = ["167_", "272_", "291_", "158_", "459_", "397_"]

# Create filesystem client
fs = AzureBlobFileSystem(
    account_name=ACCOUNT_NAME,
    account_key=ACCOUNT_KEY,
)

print("Connected to ADLS filesystem:", FILE_SYSTEM)

Connected to ADLS filesystem: scania-dataset


### Helper to read CSV from ADLS

In [2]:
def read_csv_from_adls(fs, path: str, **read_csv_kwargs) -> pd.DataFrame:
    """
    Read a CSV file from ADLS Gen2 into a pandas DataFrame.
    `path` is relative to the filesystem root, e.g. 'train/train_operational_readouts.csv'.
    """
    print(f"Loading: {path}")
    with fs.open(path, "rb") as f:
        df = pd.read_csv(f, **read_csv_kwargs)
    print(f"→ loaded shape: {df.shape}")
    return df

### Load the three training tables

In [3]:
df_oper = read_csv_from_adls(fs, TRAIN_OP_PATH)
df_tte = read_csv_from_adls(fs, TRAIN_TTE_PATH)
df_spec = read_csv_from_adls(fs, TRAIN_SPEC_PATH)

print("\nOperational columns:", df_oper.columns[:15])
print("TTE columns:", df_tte.columns)
print("Specifications columns:", df_spec.columns)

Loading: scania-dataset/train/train_operational_readouts.csv
→ loaded shape: (1122452, 107)
Loading: scania-dataset/train/train_tte.csv
→ loaded shape: (23550, 3)
Loading: scania-dataset/train/train_specifications.csv
→ loaded shape: (23550, 9)

Operational columns: Index(['vehicle_id', 'time_step', '171_0', '666_0', '427_0', '837_0', '167_0',
       '167_1', '167_2', '167_3', '167_4', '167_5', '167_6', '167_7', '167_8'],
      dtype='object')
TTE columns: Index(['vehicle_id', 'length_of_study_time_step', 'in_study_repair'], dtype='object')
Specifications columns: Index(['vehicle_id', 'Spec_0', 'Spec_1', 'Spec_2', 'Spec_3', 'Spec_4',
       'Spec_5', 'Spec_6', 'Spec_7'],
      dtype='object')


### Merge tables into time-step level dataset (code)

In [4]:
# Merge operational + TTE on (vehicle_id)
df_merged = pd.merge(
    df_oper,
    df_tte[[VEHICLE_COL, TARGET_COL, "length_of_study_time_step"]],
    on=VEHICLE_COL,
    how="left",
    validate="many_to_one",
)

print("Merged operational + TTE shape:", df_merged.shape)

# Merge specifications (static per vehicle)
df_full = pd.merge(
    df_merged,
    df_spec,
    on=VEHICLE_COL,
    how="left",
    validate="many_to_one",
)

print("Full merged shape (operational + TTE + specs):", df_full.shape)
df_full.head()

Merged operational + TTE shape: (1122452, 109)
Full merged shape (operational + TTE + specs): (1122452, 117)


,vehicle_id,time_step,171_0,666_0,427_0,837_0,167_0,167_1,167_2,167_3,...,in_study_repair,length_of_study_time_step,Spec_0,Spec_1,Spec_2,Spec_3,Spec_4,Spec_5,Spec_6,Spec_7
0,0,11.2,167985.0,10787.0,7413813.0,2296.0,4110.0,1296420.0,1628265.0,630345.0,...,0,510.0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0
1,0,11.4,167985.0,10787.0,7413813.0,2296.0,4111.0,1302855.0,1628265.0,630345.0,...,0,510.0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0
2,0,19.6,331635.0,14525.0,13683604.0,2600.0,NaN,NaN,NaN,NaN,...,0,510.0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0
3,0,20.2,354975.0,15015.0,14540449.0,2616.0,NaN,NaN,NaN,NaN,...,0,510.0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0
4,0,21.0,365550.0,15295.0,14966985.0,2720.0,NaN,NaN,NaN,NaN,...,0,510.0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0,Cat0


### Identify counter and histogram columns actually present

In [5]:
all_cols = df_full.columns.tolist()

# Keep only counters that actually exist
counter_cols_present = [c for c in COUNTER_COLS if c in all_cols]
print("Counter columns present:", counter_cols_present)

# Collect histogram columns by prefix
histogram_groups = {}
for prefix in HISTOGRAM_PREFIXES:
    cols = [c for c in all_cols if c.startswith(prefix)]
    if cols:
        histogram_groups[prefix[:-1]] = cols  # e.g. "167"
print("Histogram groups (prefix -> #cols):", {k: len(v) for k, v in histogram_groups.items()})

Counter columns present: ['171_0', '666_0', '427_0', '837_0', '309_0', '835_0', '370_0', '100_0']
Histogram groups (prefix -> #cols): {'167': 10, '272': 10, '291': 11, '158': 10, '459': 20, '397': 36}


### Helper: linear trend & R²

In [6]:
def linear_trend(x: np.ndarray, y: np.ndarray):
    """
    Fit a simple linear regression y = a*x + b.
    Returns (slope a, intercept b, r2).
    Handles constant or too-short series.
    """
    mask = np.isfinite(x) & np.isfinite(y)
    if mask.sum() < 2:
        return np.nan, np.nan, np.nan

    x = x[mask]
    y = y[mask]

    # Center x for numerical stability
    x_mean = x.mean()
    x_centered = x - x_mean

    # Fit slope & intercept
    slope, intercept = np.polyfit(x_centered, y, 1)
    intercept = intercept - slope * x_mean  # adjust intercept back

    # Compute R²
    y_pred = slope * x + intercept
    ss_res = np.sum((y - y_pred) ** 2)
    ss_tot = np.sum((y - y.mean()) ** 2)
    r2 = 1 - ss_res / ss_tot if ss_tot > 0 else np.nan

    return slope, intercept, r2

### Aggregate counter features per vehicle

In [7]:
def compute_counter_features(group: pd.DataFrame) -> pd.Series:
    """
    Compute aggregation features for counter-type columns for a single vehicle.
    Assumes group is all rows for one vehicle_id.
    """
    out = {}

    time = group[TIME_COL].values.astype(float)

    for col in counter_cols_present:
        vals = group[col].values.astype(float)

        mask = np.isfinite(vals)
        if mask.sum() == 0:
            # all NaN
            out[f"{col}_first"] = np.nan
            out[f"{col}_last"] = np.nan
            out[f"{col}_delta"] = np.nan
            out[f"{col}_mean"] = np.nan
            out[f"{col}_std"] = np.nan
            out[f"{col}_slope"] = np.nan
            out[f"{col}_r2"] = np.nan
            continue

        # Basic stats
        vals_valid = vals[mask]
        time_valid = time[mask]
        out[f"{col}_first"] = vals_valid[0]
        out[f"{col}_last"] = vals_valid[-1]
        out[f"{col}_delta"] = vals_valid[-1] - vals_valid[0]
        out[f"{col}_mean"] = vals_valid.mean()
        out[f"{col}_std"] = vals_valid.std()

        # Trend
        slope, intercept, r2 = linear_trend(time_valid, vals_valid)
        out[f"{col}_slope"] = slope
        out[f"{col}_r2"] = r2

    return pd.Series(out)


agg_counters = df_full.groupby(VEHICLE_COL).apply(compute_counter_features)
agg_counters = agg_counters.reset_index()

print("Counter feature matrix shape:", agg_counters.shape)
agg_counters.head()

Counter feature matrix shape: (23550, 57)


,vehicle_id,171_0_first,171_0_last,171_0_delta,171_0_mean,171_0_std,171_0_slope,171_0_r2,666_0_first,666_0_last,...,370_0_std,370_0_slope,370_0_r2,100_0_first,100_0_last,100_0_delta,100_0_mean,100_0_std,100_0_slope,100_0_r2
0,0,167985.0,10189950.0,10021965.0,5.310002e+06,2.950191e+06,19876.641052,0.999044,10787.0,372685.0,...,0.0,0.0,NaN,858410.0,9072889.0,8214479.0,4.778950e+06,2.184424e+06,51179.561096,0.996425
1,2,65520.0,5648790.0,5583270.0,3.374853e+06,1.582802e+06,19853.231844,0.999757,2226.0,289371.0,...,0.0,0.0,NaN,691450.0,35001073.0,34309623.0,2.237967e+07,9.441691e+06,117677.334018,0.987124
2,3,1530.0,7603590.0,7602060.0,3.274506e+06,2.457206e+06,26951.993945,0.997225,28.0,230831.0,...,0.0,0.0,NaN,65625.0,13302706.0,13237081.0,6.338494e+06,4.262806e+06,46601.496670,0.994551
3,4,210915.0,4842780.0,4631865.0,1.815351e+06,1.437392e+06,28073.986439,0.946172,21371.0,210381.0,...,0.0,0.0,NaN,1373990.0,18870423.0,17496433.0,7.263212e+06,5.131482e+06,100459.452641,0.950623
4,5,29175.0,6623040.0,6593865.0,2.570454e+06,1.876130e+06,17669.041399,0.986829,259.0,280531.0,...,0.0,0.0,NaN,152920.0,34870315.0,34717395.0,1.338433e+07,9.500105e+06,89265.637972,0.982321


### Histogram helper features (row-level)

In [8]:
def add_histogram_derived_columns(df: pd.DataFrame, histogram_groups: dict) -> pd.DataFrame:
    """
    For each histogram group (e.g., 167, 291, etc.), compute:
    - total mass per row (sum of bins)
    - centroid per row (weighted bin index)
    Adds new columns: e.g. '167_total', '167_centroid'.
    """
    df = df.copy()

    for prefix, cols in histogram_groups.items():
        bin_indices = np.arange(len(cols))

        values = df[cols].values.astype(float)
        total = np.nansum(values, axis=1)

        # Avoid division by zero
        with np.errstate(invalid="ignore", divide="ignore"):
            centroid = np.nansum(values * bin_indices, axis=1) / total

        df[f"{prefix}_total"] = total
        df[f"{prefix}_centroid"] = centroid

    return df


df_with_hist_derived = add_histogram_derived_columns(df_full, histogram_groups)
print("Columns after adding histogram-derived features:", 
      [c for c in df_with_hist_derived.columns if c.endswith("_total") or c.endswith("_centroid")])

Columns after adding histogram-derived features: ['167_total', '167_centroid', '272_total', '272_centroid', '291_total', '291_centroid', '158_total', '158_centroid', '459_total', '459_centroid', '397_total', '397_centroid']


### Aggregate histogram bins per vehicle
We’ll compute mean, std, min, max per bin, and per-vehicle mean/std for *_total and *_centroid.

In [9]:
# All histogram bin columns
hist_bin_cols = [c for cols in histogram_groups.values() for c in cols]

# Simple per-bin aggregations: mean, std, min, max per vehicle
agg_hist_bins = df_with_hist_derived.groupby(VEHICLE_COL)[hist_bin_cols].agg(["mean", "std", "min", "max"])

# Flatten column MultiIndex
agg_hist_bins.columns = [
    f"{col}_{stat}" for (col, stat) in agg_hist_bins.columns.to_flat_index()
]

agg_hist_bins = agg_hist_bins.reset_index()

print("Histogram bin feature matrix shape:", agg_hist_bins.shape)
agg_hist_bins.iloc[:3, :10]  # preview a few columns

Histogram bin feature matrix shape: (23550, 389)


,vehicle_id,167_0_mean,167_0_std,167_0_min,167_0_max,167_1_mean,167_1_std,167_1_min,167_1_max,167_2_mean
0,0,4110.500000,0.707107,4110.0,4111.0,1.299638e+06,4.550232e+03,1296420.0,1302855.0,1.628265e+06
1,2,6965.454545,2682.559448,0.0,10415.0,5.423560e+06,2.390315e+06,811605.0,9137870.0,4.780749e+07
2,3,3144.872340,2378.624535,0.0,5918.0,4.003922e+06,2.735321e+06,178155.0,8225139.0,8.405528e+06


### Aggregate histogram-derived features per vehicle

In [10]:
hist_derived_cols = [c for c in df_with_hist_derived.columns if c.endswith("_total") or c.endswith("_centroid")]

agg_hist_derived = df_with_hist_derived.groupby(VEHICLE_COL)[hist_derived_cols].agg(["mean", "std", "min", "max"])
agg_hist_derived.columns = [
    f"{col}_{stat}" for (col, stat) in agg_hist_derived.columns.to_flat_index()
]
agg_hist_derived = agg_hist_derived.reset_index()

print("Histogram-derived feature matrix shape:", agg_hist_derived.shape)
agg_hist_derived.head()

Histogram-derived feature matrix shape: (23550, 49)


,vehicle_id,167_total_mean,167_total_std,167_total_min,167_total_max,167_centroid_mean,167_centroid_std,167_centroid_min,167_centroid_max,272_total_mean,...,459_centroid_min,459_centroid_max,397_total_mean,397_total_std,397_total_min,397_total_max,397_centroid_mean,397_centroid_std,397_centroid_min,397_centroid_max
0,0,1.458139e+05,1.348263e+06,0.0,12543213.0,4.245276,0.001178,4.244443,4.246109,2.714988e+08,...,10.500424,12.059208,8.355909e+07,4.659987e+07,3380996.0,160763381.0,9.352315,0.465353,9.085742,11.768934
1,2,2.546811e+08,1.178017e+08,5970722.0,417115398.0,3.923048,0.180343,3.296193,4.029041,2.249785e+08,...,10.478325,11.557257,6.922446e+07,3.190337e+07,1599446.0,113033934.0,7.150810,0.141460,6.880720,7.337317
2,3,9.767754e+07,1.138214e+08,0.0,316522268.0,4.192523,0.765595,1.562079,4.685806,1.554991e+08,...,5.389410,12.754225,4.784659e+07,3.519508e+07,93968.0,108368001.0,7.955818,1.166679,5.480298,8.701633
3,4,7.385281e+07,8.510593e+07,0.0,254134448.0,4.824750,0.022410,4.785495,4.847020,9.783477e+07,...,9.910670,12.941943,3.010317e+07,2.331048e+07,4709433.0,78592503.0,10.841335,0.737852,10.170064,12.098348
4,5,2.539100e+06,6.190592e+06,0.0,26308377.0,3.635764,0.339751,3.229068,4.018617,1.484936e+08,...,9.259184,12.355938,4.569076e+07,3.287324e+07,525589.0,115228355.0,8.618061,0.906887,5.980907,9.929091


### Aggregate study length per vehicle
length_of_study_time_step appears once per vehicle in TTE, but we can treat it as a feature and verify.

In [11]:
# Sanity: one row per vehicle in TTE
assert df_tte[VEHICLE_COL].nunique() == len(df_tte)

study_length = df_tte[[VEHICLE_COL, "length_of_study_time_step"]].copy()
study_length.rename(columns={"length_of_study_time_step": "study_length_time_step"}, inplace=True)

study_length.head()

,vehicle_id,study_length_time_step
0,0,510.0
1,2,281.8
2,3,293.4
3,4,210.0
4,5,360.4


### Encode specifications (one-hot)

In [12]:
spec_cols = [c for c in df_spec.columns if c != VEHICLE_COL]

df_spec_encoded = pd.get_dummies(df_spec, columns=spec_cols, drop_first=True)
print("Specifications encoded shape:", df_spec_encoded.shape)
df_spec_encoded.head()

Specifications encoded shape: (23550, 83)


,vehicle_id,Spec_0_Cat1,Spec_0_Cat2,Spec_1_Cat1,Spec_1_Cat10,Spec_1_Cat11,Spec_1_Cat12,Spec_1_Cat13,Spec_1_Cat14,Spec_1_Cat15,...,Spec_6_Cat8,Spec_6_Cat9,Spec_7_Cat1,Spec_7_Cat2,Spec_7_Cat3,Spec_7_Cat4,Spec_7_Cat5,Spec_7_Cat6,Spec_7_Cat7,Spec_7_Cat8
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,3,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


### Merge all feature blocks together

In [13]:
# Start with counter features
df_features = agg_counters.copy()

# Merge histogram bin features
df_features = df_features.merge(agg_hist_bins, on=VEHICLE_COL, how="left")

# Merge histogram-derived features (total, centroid)
df_features = df_features.merge(agg_hist_derived, on=VEHICLE_COL, how="left")

# Merge study length
df_features = df_features.merge(study_length, on=VEHICLE_COL, how="left")

# Merge encoded specs
df_features = df_features.merge(df_spec_encoded, on=VEHICLE_COL, how="left")

print("Feature matrix (without target) shape:", df_features.shape)
df_features.head()

Feature matrix (without target) shape: (23550, 576)


,vehicle_id,171_0_first,171_0_last,171_0_delta,171_0_mean,171_0_std,171_0_slope,171_0_r2,666_0_first,666_0_last,...,Spec_6_Cat8,Spec_6_Cat9,Spec_7_Cat1,Spec_7_Cat2,Spec_7_Cat3,Spec_7_Cat4,Spec_7_Cat5,Spec_7_Cat6,Spec_7_Cat7,Spec_7_Cat8
0,0,167985.0,10189950.0,10021965.0,5.310002e+06,2.950191e+06,19876.641052,0.999044,10787.0,372685.0,...,0,0,0,0,0,0,0,0,0,0
1,2,65520.0,5648790.0,5583270.0,3.374853e+06,1.582802e+06,19853.231844,0.999757,2226.0,289371.0,...,0,0,1,0,0,0,0,0,0,0
2,3,1530.0,7603590.0,7602060.0,3.274506e+06,2.457206e+06,26951.993945,0.997225,28.0,230831.0,...,0,0,1,0,0,0,0,0,0,0
3,4,210915.0,4842780.0,4631865.0,1.815351e+06,1.437392e+06,28073.986439,0.946172,21371.0,210381.0,...,0,0,1,0,0,0,0,0,0,0
4,5,29175.0,6623040.0,6593865.0,2.570454e+06,1.876130e+06,17669.041399,0.986829,259.0,280531.0,...,0,0,1,0,0,0,0,0,0,0


### Add target label (in_study_repair)

In [14]:
target_df = df_tte[[VEHICLE_COL, TARGET_COL]].copy()

df_features = df_features.merge(target_df, on=VEHICLE_COL, how="left")

# Final sanity checks
print("Final feature matrix shape:", df_features.shape)
print("Number of vehicles:", df_features[VEHICLE_COL].nunique())
print("Target value counts:")
print(df_features[TARGET_COL].value_counts(dropna=False))

df_features.head()

Final feature matrix shape: (23550, 577)
Number of vehicles: 23550
Target value counts:
0    21278
1     2272
Name: in_study_repair, dtype: int64


,vehicle_id,171_0_first,171_0_last,171_0_delta,171_0_mean,171_0_std,171_0_slope,171_0_r2,666_0_first,666_0_last,...,Spec_6_Cat9,Spec_7_Cat1,Spec_7_Cat2,Spec_7_Cat3,Spec_7_Cat4,Spec_7_Cat5,Spec_7_Cat6,Spec_7_Cat7,Spec_7_Cat8,in_study_repair
0,0,167985.0,10189950.0,10021965.0,5.310002e+06,2.950191e+06,19876.641052,0.999044,10787.0,372685.0,...,0,0,0,0,0,0,0,0,0,0
1,2,65520.0,5648790.0,5583270.0,3.374853e+06,1.582802e+06,19853.231844,0.999757,2226.0,289371.0,...,0,1,0,0,0,0,0,0,0,0
2,3,1530.0,7603590.0,7602060.0,3.274506e+06,2.457206e+06,26951.993945,0.997225,28.0,230831.0,...,0,1,0,0,0,0,0,0,0,0
3,4,210915.0,4842780.0,4631865.0,1.815351e+06,1.437392e+06,28073.986439,0.946172,21371.0,210381.0,...,0,1,0,0,0,0,0,0,0,0
4,5,29175.0,6623040.0,6593865.0,2.570454e+06,1.876130e+06,17669.041399,0.986829,259.0,280531.0,...,0,1,0,0,0,0,0,0,0,0


### Basic quality checks

In [15]:
# Check for remaining missing values
missing_frac = df_features.isna().mean().sort_values(ascending=False)
print("Top 20 features by missing fraction:")
missing_frac.head(20)

Top 20 features by missing fraction:


370_0_r2             0.639108
309_0_r2             0.079745
837_0_r2             0.018047
291_centroid_std     0.000255
666_0_r2             0.000212
291_centroid_max     0.000212
291_centroid_min     0.000212
291_centroid_mean    0.000212
167_9_std            0.000127
167_2_std            0.000127
167_7_std            0.000127
167_0_std            0.000127
167_6_std            0.000127
167_1_std            0.000127
167_5_std            0.000127
167_3_std            0.000127
167_8_std            0.000127
167_4_std            0.000127
167_centroid_std     0.000127
427_0_slope          0.000085
dtype: float64

At this point you can decide:

* fill NaNs (e.g., with per-feature median) for modeling, or

* keep them and let XGBoost handle them (it can).

### Save the feature matrix

For now, save to the notebook’s local filesystem; later we can push to ADLS or create a Data Asset.

In [16]:
output_path = "train_vehicle_features.csv"
df_features.to_csv(output_path, index=False)
print(f"Saved feature matrix to: {output_path}")
print("Shape:", df_features.shape)

Saved feature matrix to: train_vehicle_features.csv
Shape: (23550, 577)
